In [1]:
import os
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
from pathlib import Path
import boto3
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

s3 = boto3.client("s3")

In [2]:
bucket = "ml-for-bem"
experiment_type = "validation"
version = "v5"
experiment_name = f"{experiment_type}/{version}"
local_dir = Path("data") / "temp" / experiment_name
redownload = False

In [3]:
def dl(file, force=False):
    os.makedirs(local_dir, exist_ok=True)
    if not (local_dir / file).exists() or force:
        print(f"Downloading {file}")
        s3.download_file(bucket, f"{experiment_name}/{file}", local_dir / file)
    else:
        print(f"File {file} already exists, skipping.")


def dl_and_open_hdf(name, key="results", force=False):
    dl(f"{name}.hdf", force=force)
    df = pd.read_hdf(local_dir / f"{name}.hdf", key=key)
    if len(df) != 10000:
        print(f"WARNING: {name} has {len(df)} rows, expected 10000")
    return df


city_map = {
    "name": [
        "Mumbai",
        "Caracas",
        "Miami",
        "Riyadh",
        "Sao Paulo",
        "Rome",
        "Tokyo",
        "Las Vegas",
        "London",
        "Seattle",
        "Krakow",
        "Oslo",
        "Fairbanks",
    ],
    "keys": [
        "mumbai",
        "caracas",
        "miami",
        "riyadh",
        "paulo",
        "rome",
        "tokyo",
        "vegas",
        "london",
        "seattle",
        "krakow",
        "oslo",
        "fairbanks",
    ],
    "Climate Zone": [
        "0A",
        "0B",
        "1B",
        "1A",
        "2A",
        "3A",
        "3A",
        "3B",
        "4A",
        "4C",
        "5A",
        "6A",
        "8",
    ],
}
city_df = pd.DataFrame(city_map)
city_df = city_df.set_index("keys")
cities = city_df.index.values
city_df["Display Name"] = city_df["name"] + " (" + city_df["Climate Zone"] + ")"

In [4]:
with ThreadPoolExecutor() as executor:
    dl_and_open = lambda x: dl_and_open_hdf(x, force=redownload)
    dfs = executor.map(dl_and_open, city_df.index.to_list())
    results = pd.concat(dfs, axis=1, keys=city_df.name.to_list())
    results = results.swaplevel(0, 1, axis=1)
    results.columns.names = ["Segment", "City", "End Use", "Month"]
    results.index.name = "building_id"
    results = results.reindex(range(len(results)))

building_features = pd.read_hdf(local_dir / "features.hdf", key="buildings")
building_features = building_features.reindex(results.index)

File mumbai.hdf already exists, skipping.
File caracas.hdf already exists, skipping.
File miami.hdf already exists, skipping.
File riyadh.hdf already exists, skipping.
File paulo.hdf already exists, skipping.
File rome.hdf already exists, skipping.
File vegas.hdf already exists, skipping.
File tokyo.hdf already exists, skipping.
File london.hdf already exists, skipping.
File seattle.hdf already exists, skipping.
File krakow.hdf already exists, skipping.
File oslo.hdf already exists, skipping.
File fairbanks.hdf already exists, skipping.


# Building Level Annual Analysis


In [5]:
annual = results.T.groupby(level=[0, 1, 2]).sum().T
true_mean = annual["True"].mean()
annual_rmse = np.sqrt(annual["Residuals"].pow(2).mean())
annual_cvrmse = annual_rmse / true_mean * 100
annual_mae = annual["Residuals"].abs().mean()
annual_mbe = annual["Residuals"].mean()
annual_mape = (annual["Residuals"].abs() / annual["True"] * 100).mean()
annual_summary = pd.concat(
    [
        true_mean.rename("True Mean [kWh/m2]"),
        annual_rmse.rename("RMSE [kWh/m2]"),
        annual_cvrmse.rename("CVRMSE"),
        annual_mae.rename("MAE [kWh/m2]"),
        annual_mbe.rename("MBE [kWh/m2]"),
    ],
    axis=1,
).drop("Electricity", level=-1)
annual_summary.round(2)

True Mean [kWh/m2]  RMSE [kWh/m2]   CVRMSE  MAE [kWh/m2]  \
City      End Use                                                             
Caracas   Cooling              108.62          20.04    18.45         15.13   
          Heating                0.09           0.58   617.14          0.26   
Fairbanks Cooling               11.33           5.45    48.09          2.96   
          Heating              208.89          33.33    15.95         26.33   
Krakow    Cooling               17.39           5.90    33.95          3.10   
          Heating               78.51          16.09    20.50         11.81   
Las Vegas Cooling              101.99          13.63    13.36          9.63   
          Heating               18.96           9.01    47.52          6.34   
London    Cooling               14.13           6.25    44.25          3.09   
          Heating               51.11          13.67    26.76          9.68   
Miami     Cooling              150.76          23.02    15.27         18.25   
          Heating                0.95           1.48   155.31          0.80   
Mumbai    Cooling              235.26          39.74    16.89         32.37   
          Heating                0.01           0.29  2428.07          0.26   
Oslo      Cooling                9.63           5.56    57.73          2.64   
          Heating               84.64          17.40    20.56         12.20   
Riyadh    Cooling              167.08          18.52    11.08         13.77   
          Heating                3.30           2.73    82.84          1.66   
Rome      Cooling               39.25           8.60    21.92          5.14   
          Heating               28.88          11.00    38.07          7.85   
Sao Paulo Cooling               63.31          10.56    16.68          7.29   
          Heating                2.70           2.60    96.30          1.23   
Seattle   Cooling               20.38           6.60    32.38          3.51   
          Heating               48.45          14.89    30.74         10.75   
Tokyo     Cooling               55.90           8.48    15.16          5.56   
          Heating               29.11           8.75    30.07          5.96   

                   MBE [kWh/m2]  
City      End Use                
Caracas   Cooling        -13.64  
          Heating          0.25  
Fairbanks Cooling          0.33  
          Heating         17.49  
Krakow    Cooling          0.38  
          Heating          7.45  
Las Vegas Cooling         -8.45  
          Heating          5.76  
London    Cooling          0.60  
          Heating          6.84  
Miami     Cooling        -17.79  
          Heating          0.76  
Mumbai    Cooling        -32.12  
          Heating          0.25  
Oslo      Cooling          1.31  
          Heating          7.40  
Riyadh    Cooling        -12.80  
          Heating          1.53  
Rome      Cooling         -2.75  
          Heating          6.58  
Sao Paulo Cooling         -3.68  
          Heating          1.05  
Seattle   Cooling         -0.14  
          Heating          8.67  
Tokyo     Cooling         -3.49  
          Heating          4.11

In [6]:
annual_summary["Climate Zone"] = (
    city_df.set_index("name")
    .loc[annual_summary.index.get_level_values(0), "Climate Zone"]
    .values
)
annual_summary = annual_summary.sort_values("Climate Zone")
annual_summary_p = annual_summary.reset_index()
annual_summary_p["City"] = (
    annual_summary_p["City"] + " (" + annual_summary_p["Climate Zone"] + ")"
)
annual_summary_p = annual_summary_p.reset_index()
fig = px.bar(
    # annual_summary_p[annual_summary_p["CVRMSE"] < 100],
    annual_summary_p,
    title="Building Energy Model Validation, CVRMSE",
    x="City",
    y="RMSE [kWh/m2]",
    color="End Use",
    color_discrete_map={"Heating": "#EF553B", "Cooling": "#636EFA"},
    barmode="group",
    height=600,
    width=800,
)
# fig.write_image("figures/rmse_by_city.png", scale=2)
fig

# UBEM Level Annual Analysis w/ COPs & Elec


In [7]:
np.random.seed(12)
ubem_sims = pd.DataFrame()
n_sims = 1
for i in range(n_sims):
    ubem = annual.swaplevel(0, 2, axis=1).swaplevel(1, 2, axis=1).sort_index(axis=1)
    elec_heating_p = 0.1
    is_elec_heating = np.random.rand(len(ubem)) < elec_heating_p
    heating_cop = (
        is_elec_heating * np.random.rand(len(ubem)) * 2
        + 2
        + (1 - is_elec_heating) * (0.92 + np.random.rand(len(ubem)) * 0.05)
    )
    cooling_cop = np.random.rand(len(ubem)) * 2 + 2
    ubem["Heating"] = ubem["Heating"] / heating_cop.reshape(-1, 1)
    ubem["Cooling"] = ubem["Cooling"] / cooling_cop.reshape(-1, 1)
    # ubem =ubem.drop("Electricity",axis=1)

    # aggregate to total building normalized energy
    ubem = ubem.T.groupby(level=[1, 2]).sum().T

    # ubem is in kWh/m2, so before summing we need to multiply by GFAs
    ubem = building_features.gfa.values.reshape(-1, 1) * ubem
    ubem = ubem.sum() / building_features.gfa.sum()
    if len(ubem_sims) == 0:
        ubem_sims = pd.DataFrame(ubem).T
        ubem_sims = ubem_sims.reindex(range(n_sims))
    else:
        ubem_sims.iloc[i] = ubem

error = (ubem_sims["Predicted"] - ubem_sims["True"]) / ubem_sims["True"] * 100
error_std = error.std()
error = error.abs().mean()
predicted = ubem_sims["Predicted"].mean()
true = ubem_sims["True"].mean()

a = pd.DataFrame(error, columns=["% Error"])
b = pd.DataFrame(predicted, columns=["Predicted"])
c = pd.DataFrame(true, columns=["True"])
ubem = pd.concat([a, b, c], axis=1).reset_index()

ubem["Climate Zone"] = (
    city_df.set_index("name").loc[ubem["City"]]["Climate Zone"].values
)
ubem.sort_values("Climate Zone", inplace=True)
ubem["City"] = city_df.set_index("name").loc[ubem["City"]]["Display Name"].values

px.bar(
    ubem,
    x="City",
    y="% Error",
    title=f"UBEM Error",
    range_y=[0, 15],
)

## UBEM Annual Analysis w/ COPs (no electricity) Breakdown Viz


In [8]:
ubem = annual.swaplevel(0, 2, axis=1).swaplevel(1, 2, axis=1).sort_index(axis=1)
elec_heating_p = 0.1
is_elec_heating = np.random.rand(len(ubem)) < elec_heating_p
heating_cop = (
    is_elec_heating * np.random.rand(len(ubem)) * 2
    + 2
    + (1 - is_elec_heating) * (0.92 + np.random.rand(len(ubem)) * 0.05)
)
cooling_cop = np.random.rand(len(ubem)) * 2 + 2
ubem["Heating"] = ubem["Heating"] / heating_cop.reshape(-1, 1)
ubem["Cooling"] = ubem["Cooling"] / cooling_cop.reshape(-1, 1)
# ubem =ubem.drop("Electricity",axis=1)

# # aggregate to total building normalized energy
# ubem = ubem.T.groupby(level=[1, 2]).sum().T

# ubem is in kWh/m2, so before summing we need to multiply by GFAs
ubem = building_features.gfa.values.reshape(-1, 1) * ubem
ubem = ubem.swaplevel(0, 1, axis=1).sum() / building_features.gfa.sum()

ubem = pd.DataFrame(ubem, columns=["Energy [kWh/m2]"])

ubem = ubem.drop("Electricity", level=1)
ubem = ubem.drop("Residuals").reset_index()
ubem["Climate Zone"] = (
    city_df.set_index("name").loc[ubem["City"]]["Climate Zone"].values
)
ubem = ubem.sort_values(["Climate Zone", "Segment"])
ubem["City"] = (
    city_df.set_index("name").loc[ubem["City"]]["Display Name"].values
    + " ["
    + ubem["Segment"]
    + "]"
)


px.bar(
    ubem,
    x="City",
    y="Energy [kWh/m2]",
    barmode="stack",
    color="End Use",
    color_discrete_map={
        "Heating": "#EF553B",
        "Cooling": "#636EFA",
        "Electricity": "#00CC96",
    },
)

C:\Users\szvsw\AppData\Local\Temp\ipykernel_7776\516248586.py:24: PerformanceWarning:

dropping on a non-lexsorted multi-index without a level parameter may impact performance.



# Building-Level Global Annual Analysis by End Use


In [53]:
world = annual.stack(level=1)
world_means = world["True"].mean()
world_rmse = np.sqrt(world["Residuals"].pow(2).mean())
world_cvrmse = world_rmse / world_means * 100
world_mae = world["Residuals"].abs().mean()
world_mbe = world["Residuals"].mean()
world_mape = (world["Residuals"].abs() / world["True"] * 100).mean()

world_cvrmse

End Use
Cooling        21.359046
Electricity     0.032991
Heating        31.636896
dtype: float64

# Building-Level Global Annual Analysis with COPs and Elec


In [52]:
world = annual.stack(level=1)
world = pd.concat([world for _ in range(10)], axis=0)
elec_heating_p = 0.1
is_elec_heating = np.random.rand(len(world)) < elec_heating_p
heating_cop = (
    is_elec_heating * np.random.rand(len(world)) * 2
    + 2
    + (1 - is_elec_heating) * (0.92 + np.random.rand(len(world)) * 0.05)
)
cooling_cop = np.random.rand(len(world)) * 2 + 2
world = world.swaplevel(0, 1, axis=1).sort_index(axis=1)
world["Cooling"] = world["Cooling"] / cooling_cop.reshape(-1, 1)
world["Heating"] = world["Heating"] / heating_cop.reshape(-1, 1)
world = world.swaplevel(0, 1, axis=1).sort_index(axis=1)
world = world.T.groupby("Segment").sum().T
assert (world.Residuals + (world.Predicted - world["True"])).abs().max() < 1e-6
world_building_true_mean = world["True"].mean()
world_building_rmse = np.sqrt(world["Residuals"].pow(2).mean())
world_building_cvrmse = world_building_rmse / world_building_true_mean * 100
world_building_mae = world["Residuals"].abs().mean()
world_building_mbe = world["Residuals"].mean()
world_building_mape = (world["Residuals"].abs() / world["True"] * 100).mean()

print("---Global Stats---")
print("Individual Building CVRMSE:        ", round(world_building_cvrmse, 2))
print("Individual Building RMSE (kWh/m2): ", round(world_building_rmse, 2))
print("Individual Building MBE (kWh/m2):  ", round(world_building_mbe, 2))
print("Individual Building MAE (kWh/m2):  ", round(world_building_mae, 2))

world = world.groupby(["City", "building_id"]).mean()
assert (world.Residuals + (world.Predicted - world["True"])).abs().max() < 1e-6
buildings_by_city = world.unstack(level=0)
buildings_by_city_true_mean = buildings_by_city["True"].mean()
buildings_by_city_rmse = np.sqrt(buildings_by_city["Residuals"].pow(2).mean())
buildings_by_city_cvrmse = buildings_by_city_rmse / buildings_by_city_true_mean * 100
buildings_by_city_mae = buildings_by_city["Residuals"].abs().mean()
buildings_by_city_mbe = buildings_by_city["Residuals"].mean()

buildings_by_city_summary = pd.concat(
    [
        buildings_by_city_true_mean.rename("True Mean [kWh/m2]"),
        buildings_by_city_rmse.rename("RMSE [kWh/m2]"),
        buildings_by_city_cvrmse.rename("CVRMSE"),
        buildings_by_city_mae.rename("MAE [kWh/m2]"),
        buildings_by_city_mbe.rename("MBE [kWh/m2]"),
    ],
    axis=1,
)

buildings_by_city_summary["Climate Zone"] = (
    city_df.set_index("name")
    .loc[buildings_by_city_summary.index.get_level_values(0), "Climate Zone"]
    .values
)
buildings_by_city_summary = buildings_by_city_summary.sort_values("Climate Zone")
buildings_by_city_summary.round(2)

---Global Stats---
Individual Building CVRMSE:         5.56
Individual Building RMSE (kWh/m2):  7.09
Individual Building MBE (kWh/m2):   -0.7
Individual Building MAE (kWh/m2):   4.82


,True Mean [kWh/m2],RMSE [kWh/m2],CVRMSE,MAE [kWh/m2],MBE [kWh/m2],Climate Zone
City,,,,,,
Mumbai,167.87,13.73,8.18,11.15,-11.06,0A
Caracas,124.00,6.91,5.57,5.20,-4.67,0B
Riyadh,145.42,6.09,4.19,4.45,-3.94,1A
Miami,138.91,7.81,5.62,6.12,-5.93,1B
Sao Paulo,109.21,3.60,3.30,2.52,-0.95,2A
Rome,109.77,3.99,3.64,2.90,1.26,3A
Tokyo,115.66,3.54,3.06,2.45,0.16,3A
Las Vegas,128.17,4.24,3.31,2.99,-0.99,3B
London,108.66,4.89,4.50,3.61,2.51,4A


In [57]:
buildings_by_city_summary_p = buildings_by_city_summary.reset_index()
buildings_by_city_summary_p["City"] = (
    buildings_by_city_summary_p["City"]
    + " ("
    + buildings_by_city_summary_p["Climate Zone"]
    + ")"
)

fig = px.bar(
    buildings_by_city_summary_p,
    title="Building Total Energy, CVRMSE",
    x="City",
    y="CVRMSE",
    barmode="group",
    height=600,
    width=800,
    range_y=[0, 15],
)
fig